In [113]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from preprocessing_helpers import *
from Basics import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
DATA_TRAIN_PATH = 'train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)


In [3]:
print(np.unique(y, return_counts=True))

(array([-1.,  1.]), array([164333,  85667], dtype=int64))


In [4]:
def jet_number(x, y, id):
    ind = []
    yy = []
    xx = []
    iid = []
    for n in range(4):
        ind.append(np.where(tX[:, 22]==n)[0])
        yy.append(Y[ind[n]])
        xx.append(tX[ind[n], :])
        iid.append(ids[ind[n]])
    return yy, xx, iid
y_list, X_list, iid = jet_number(tX, Y, ids)
print(y_list[2])

NameError: name 'Y' is not defined

In [166]:
# split test-train set
indx = np.arange(y.shape[0])
np.random.shuffle(indx)
ind_max = int(y.shape[0] * (0.8))

trainY , trainF = y[indx[:ind_max]] , tX[indx[:ind_max]]
testY , testF = y[indx[ind_max:]] , tX[indx[ind_max:]]
print(trainY.shape)

(200000,)


In [167]:
####### Preprocessing ######################################################################


In [168]:
# replace -999
trainF, testF = replace_missing_values(trainF, testF, 'mode')


In [169]:
# replace outliers
trainF = outliers_removal(trainF)
testF = outliers_removal(testF)

In [170]:
# normalize
trainF, testF = standardize(trainF, testF)
trainF = np.insert(trainF ,0 , np.ones(trainF.shape[0]),axis = 1) # add bias
testF= np.insert(testF ,0 , np.ones(testF.shape[0]),axis = 1) # add bias

In [171]:
# dataset balencing
trainF , trainY =  under_over(trainF, trainY, alpha=0.95, upsample=True, middle=False, gaussian=True, std=0.05, downsample=False)
print(np.unique(trainY, return_counts=True))
print(trainY.shape)

(array([-1.,  1.]), array([131468, 133637], dtype=int64))
(265105,)


In [172]:
# feature selection expansion
trainF = select_and_expand_f(trainF)
testF = select_and_expand_f(testF)

In [13]:
###### Training/Testing ######################################################################

In [264]:
w_initial = np.random.rand(trainF.shape[1])

w,_ = least_squares_GD( trainY,trainF ,w_initial,2000,0.01)

get_accuracy(w , testY ,testF )

0.8292048241004855

In [263]:
w_initial = np.random.rand(trainF.shape[1])
w,_ = least_squares(trainY,trainF)
get_accuracy(w , testY ,testF )

LinAlgError: Singular matrix

In [84]:
#A = [3, 8, 14, 2, 10, 23, 12, 4, 9, 11, 17, 20, 22, 18, 15]  # selected features through feature weight analysis
A = [range(trainF.shape[1])]
trainF = np.column_stack([trainF[:, a] for a in A])   # stacks the feature columns together for train set
#x_train = np.column_stack((np.ones(x_train.shape[0]), x_train))
testF = np.column_stack([testF[:, a] for a in A]) # stacks the feature columns together for test set
#x_test = np.column_stack((np.ones(x_test.shape[0]), x_test))
trainY = trainY[:]    # copies train labels
testY = testY[:] # copies test labels


trainF = np.column_stack([trainF**i for i in (1, 2)])  # polynomial feature multiplicattion for train
testF = np.column_stack([testF**i for i in (1, 2)])    # polynomial feature multiplicattion for test
print(trainF.shape)


(79930, 64)


In [173]:
w = np.random.rand(trainF.shape[1])
w, losses, _, _, _, _ = foo_logistic_regression(trainY, trainF, w, 1000, 0.07, testY, testF, 0)


Progress=99.9%, Accuracy=77.092%

In [174]:
TP, FP, TN, FN = logistic_accuracy(testY, testF, w)
#P = 1
#N = -1
TPR = 100*TP/(TP + FN)
TNR = 100*TN/(TN + FP)
PPV = 100*TP/(TP + FP)
NPV = 100*TN/(TN + FN)
F1 = 2*(TPR*PPV)/(TPR+PPV)
print(TPR, TNR, PPV, NPV, '\n', F1)

63.471662319184254 87.02403596749092 78.10329734461628 76.56473452000608 
 70.03139717425432


In [86]:
w = np.random.rand(trainF.shape[1])
w, _ = logistic_regression(trainY, trainF, w, 500,0.07)

get_accuracy_logistic(w , testY ,testF)

KeyboardInterrupt: 

In [18]:
def predict(w, x, y, id):
    xx, yy, iid = jet_number(x, y, id)
    y_pred = []
    for n in range(4):
        y_pred.append(predict_logistic_labels(xx[n], w[0]))
    
    y_pred = np.vstack([y_pred[n] for n in range(4)])
    ids = np.vstack([iid[n] for n in range(4)])
    return y_pred, ids